In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time

df = pd.read_csv('../uncleaned_datasets/data_fnb_bandung.csv')

df = df.head(10)

df.head()

,id,kode_provinsi,nama_provinsi,bps_kode_kabupaten_kota,bps_nama_kabupaten_kota,nama_rumah_makan,alamat,tahun
0,1,32,JAWA BARAT,3273,KOTA BANDUNG,BREW & CHEW,JL. SUKABUMI NO.53,2020
1,2,32,JAWA BARAT,3273,KOTA BANDUNG,CAFE BALI,JL. L. L. R. E. MARTADINATA NO.211-215,2020
2,3,32,JAWA BARAT,3273,KOTA BANDUNG,ANJIS COFFEE,JL. BENGAWAN NO. 34,2020
3,4,32,JAWA BARAT,3273,KOTA BANDUNG,HEADKLIN BARBER & COFFEE HOUSE,JL. CIANJUR NO.12A,2020
4,5,32,JAWA BARAT,3273,KOTA BANDUNG,GARDEN COFFEE BDG,JL. BENGAWAN NO.7A,2020


In [44]:
streets = df['alamat']
streets.isnull().sum()

np.int64(0)

In [45]:
def get_area_details_by_place_and_street(street_name):
    """
    Mendapatkan detail kelurahan dan kecamatan berdasarkan nama tempat dan nama jalan.
    """
    # Penting: Ganti 'my_combined_search_app' dengan nama aplikasi Anda yang unik
    geolocator = Nominatim(user_agent="my_combined_search_app")

    # Gabungkan nama tempat dan nama jalan untuk membuat alamat yang lebih spesifik
    # Anda bisa menambahkan kota/provinsi/negara jika dirasa perlu untuk akurasi yang lebih tinggi
    full_address = f"{street_name}, Bandung, West Java, Indonesia"

    try:
        # Lakukan geocoding
        # addressdetails=True sangat penting untuk mendapatkan detail komponen alamat
        location = geolocator.geocode(full_address, addressdetails=True, language='id')

        if location:
            address_components = location.raw.get('address', {})
            
            # Mendapatkan Kelurahan/Desa
            # Cek berbagai kemungkinan kunci yang digunakan Nominatim
            kelurahan = address_components.get('village') or \
                        address_components.get('suburb') or \
                        address_components.get('locality') or \
                        address_components.get('hamlet') # 'hamlet' bisa jadi opsi lain untuk desa kecil

            # Mendapatkan Kecamatan
            # Cek berbagai kemungkinan kunci yang digunakan Nominatim
            kecamatan = address_components.get('county') or \
                        address_components.get('state_district') or \
                        address_components.get('city_district') or \
                        address_components.get('town')

            # print(f"Pencarian untuk: '{street_name}' di '{place_name}'")
            print(f"Alamat Lengkap Ditemukan: {location.address}")
            print(f"Latitude: {location.latitude}")
            print(f"Longitude: {location.longitude}")
            print(f"Kelurahan/Desa: {kelurahan if kelurahan else 'Tidak ditemukan'}")
            print(f"Kecamatan: {kecamatan if kecamatan else 'Tidak ditemukan'}")
            print("-" * 40)

            data = [kelurahan, kecamatan, location.latitude, location.longitude]
            return pd.Series(data, index=['kelurahan', 'kecamatan', 'latitude', 'longitude'])

        else:
            print(f"Tidak dapat menemukan detail lokasi untuk: '{street_name}'")
            print("-" * 40)

    except GeocoderTimedOut:
        print("Error: Layanan geocoding time out. Silakan coba lagi nanti.")
    except GeocoderServiceError as e:
        print(f"Error dari layanan geocoding: {e}")
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

In [ ]:
df[['kelurahan', 'kecamatan']] = df.apply(
    lambda row: get_area_details_by_place_and_street(row['alamat']),
    axis=1
)

df.head()

# Simpan hasil ke file CSV baru
df.to_csv('../datasets/data_fnb_bandung_cleaned.csv', index=False)
print("Data telah disimpan ke 'data_fnb_bandung_cleaned.csv'.")

Alamat Lengkap Ditemukan: Jalan Sukabumi, Kacapiring, Batununggal, Kota Bandung, Jawa Barat, Jawa, 40271, Indonesia
Latitude: -6.9178677
Longitude: 107.6329391
Kelurahan/Desa: Kacapiring
Kecamatan: Tidak ditemukan
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: 'JL. L. L. R. E. MARTADINATA NO.211-215'
----------------------------------------
Alamat Lengkap Ditemukan: Jalan Bengawan, Cihapit, Bandung Wetan, Kota Bandung, Jawa Barat, Jawa, 40121, Indonesia
Latitude: -6.9119978
Longitude: 107.6321563
Kelurahan/Desa: Cihapit
Kecamatan: Tidak ditemukan
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: 'JL. CIANJUR NO.12A'
----------------------------------------
Tidak dapat menemukan detail lokasi untuk: 'JL. BENGAWAN NO.7A'
----------------------------------------


KeyboardInterrupt: 